In [9]:
import os


import urllib.request as request
import zipfile

from pathlib import Path
from dataclasses import dataclass

from ImageClassification import logger
from ImageClassification.utils.common import get_size

from ImageClassification.utils.common import read_yaml, create_directories
from ImageClassification.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

In [2]:
'''
Setting up the directories
'''

CURRENT_DIRECTORY = os.getcwd()
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)

# data Directory
DATA_PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)
DATA_DIRECTORY = os.path.join(DATA_PARENT_DIRECTORY, '/_data_DL_Chicken_Disease_Classification_AWS_AZURE_DVC')

TRAIN_DATA = os.path.join(DATA_PARENT_DIRECTORY, '/train')
VALIDATION_DATA = os.path.join(DATA_PARENT_DIRECTORY, '/test')
TEST_DATA = os.path.join(DATA_PARENT_DIRECTORY, '/validation')

# # # Usage
# CONFIG_DIRECTORY = os.path.join(PARENT_DIRECTORY, 'config')
# CONFIG_FILE_PATH = os.path.join(CONFIG_DIRECTORY, 'config.yaml')


# print('CURRENT_DIRECTORY', CURRENT_DIRECTORY)
# print('PARENT_DIRECTORY', PARENT_DIRECTORY)
# print('DATA_PARENT_DIRECTORY', DATA_PARENT_DIRECTORY)
# print('DATA_DIRECTORY', DATA_DIRECTORY)
# print('TRAIN_DATA', TRAIN_DATA)
# print('VALIDATION_DATA', VALIDATION_DATA)
# print('TEST_DATA', TEST_DATA)
# print('CONFIG_DIRECTORY', CONFIG_DIRECTORY)
# print('CONFIG_FILE_PATH', CONFIG_FILE_PATH)

In [3]:


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
class ConfigurationManager:
    def __init__(self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [5]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [6]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Error during data ingestion: {e}")
    raise e


YAML content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1xi8NeALtgToc2GONJuv1AoTwTY62RPVg/view?usp=drive_link', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2024-05-21 18:23:18,148: INFO: common: YAML file: C:\Users\Nimbus\_00_Peter_Practice_Projects\_02_In_Progress\_02_ML_Projects_Krish_Naik_Course_list\_02_DL_MLOPS_DVC_AZURE_AWS\DL_Chicken_Disease_Classification_AWS_AZURE_DVC\config\config.yaml loaded successfully]
YAML content: {'learning_rate': 0.01, 'batch_size': 32, 'num_epochs': 10}
[2024-05-21 18:23:18,150: INFO: common: YAML file: C:\Users\Nimbus\_00_Peter_Practice_Projects\_02_In_Progress\_02_ML_Projects_Krish_Naik_Course_list\_02_DL_MLOPS_DVC_AZURE_AWS\DL_Chicken_Disease_Classification_AWS_AZURE_DVC\params.yaml loaded successfully]
[2024-05-21 18:23:18,151: INFO: common: Created Directory at : artifacts]
[2024-05-21 18:23:18,15

In [8]:
# -----------------------------------